---
# --- Day 14: Parabolic Reflector Dish ---
---

In [1]:
import numpy as np
from typing import List

In [2]:
V = lambda x: np.array(x)

## Load data

In [3]:
full_puzzle_data = True

In [4]:
def conv_to_int(c: str) -> int:
    if c == "#":
        return -1
    elif c == "O":
        return 1
    else:
        return 0

In [5]:
def get_input_data(full_puzzle_data: bool):
    file_suffix = "" if full_puzzle_data else "_test"
    with open(f"data/day14_input{file_suffix}.txt", "r") as f:
        data = V([list(map(conv_to_int, list(r))) for r in f.read().splitlines()])
    return data

In [6]:
platform = get_input_data(full_puzzle_data)

In [7]:
def print_platform(ptf: np.ndarray) -> None:
    chars = ["#", ".", "O"]
    for i in range(len(ptf)):
        line = "".join([chars[j+1] for j in ptf[i, :]])
        print(line)

In [8]:
if not full_puzzle_data:
    print_platform(platform)

## --- Part One ---

In [9]:
def slide_ones(vec: np.ndarray) -> List[int]:
    
    hash_pos = [-1] + list(np.where(vec==-1)[0]) + [len(vec)]
    out_vec = []
    for i in range(len(hash_pos) - 1):
        n_ones = sum(vec[hash_pos[i]+1:hash_pos[i+1]])
        segm = [1]*n_ones + [0]*(hash_pos[i+1]-hash_pos[i]-1-n_ones) + [-1]
        out_vec += segm
    
    return out_vec[:-1]

In [10]:
def compute_north_load(ptf: np.ndarray) -> int:
    m, n = ptf.shape
    ptf = np.clip(ptf, 0, 1)
    weights = np.tile(V(range(m, 0, -1)).reshape(-1,1), (1,n))
    return np.multiply(ptf, weights).sum()

In [11]:
slided_platform = platform.copy()
for j in range(len(slided_platform)):
    slided_platform[:, j] = slide_ones(slided_platform[:, j])

In [12]:
print(compute_north_load(slided_platform))

103614


## --- Part Two ---

In [13]:
def do_one_cycle(ptf_start: np.ndarray) -> np.ndarray:
    ptf = ptf_start.copy()
    m, n = ptf.shape
    for j in range(m): # north
        ptf[:, j] = slide_ones(ptf[:, j])
    for i in range(n): # west
        ptf[i, :] = slide_ones(ptf[i, :])
    for j in range(m): # south
        ptf[:, j] = slide_ones(ptf[:, j][::-1])[::-1]
    for i in range(n): # east
        ptf[i, :] = slide_ones(ptf[i, :][::-1])[::-1]
    return ptf

In [14]:
slided_platform = platform.copy()
last_cycle = -1
outputs = []
while last_cycle < 200:
    last_cycle += 1
    #print(f"Cycle <{cycle_count+1}>")
    slided_platform = do_one_cycle(slided_platform)
    outputs.append(slided_platform)
    if last_cycle != 0:
        check = [np.array_equal(slided_platform, p) for p in outputs[:-1]]
        if any(check):
            first_cycle = np.argmax(check)
            print(f"Recurrence found between cycle {first_cycle+1} and {last_cycle+1}!")
            break

Recurrence found between cycle 103 and 175!


In [15]:
n = 1000000000
period = last_cycle - first_cycle
n_cycle_in_period = (n - 1 - first_cycle) % period
same_platform = outputs[first_cycle + n_cycle_in_period]
print(compute_north_load(same_platform))

83790
